## The way this code is written, catalogs need to be set up before running this code. For both output catalogs, I created dummy versions where the contents are all labeled testtest and all numeric values are set to -99.99. After the code is ran, it updates the rows with correct information.

In [1]:
import configparser
import glob,os,sys
import time
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table
from astropy.table import QTable
from photutils import aperture_photometry
from astropy.coordinates import SkyCoord  # High-level coordinates
import astropy.units as u
from photutils import SkyCircularAperture
import pyregion

from HST_filters import HST_filt  # This will point to the HST throughput files

In [2]:
####################################
# constants
####################################
c = 2.9979E10       # cm/s
Ang = 1E-8          # cm
Jy = 1.0E-23        # erg/s/cm^2/Hz
mJy = 1e-3          # Jy
uJy = 1e-6          # Jy
Mpc = 3.086e24      # cm
## PHOTFLAM: inverse sensitivity (erg * cm**(-2) * s**(-1) Angstrom**(-1)
## PHOTPLAM: Pivot wavelength

In [3]:
def hst_phot(photflam,photplam,dn,dn_err=None,unit="Jy"):
    ## http://www.stsci.edu/hst/acs/analysis/zeropoints
    #ABMAG_ZP=-2.5*np.log10(photflam)-5*np.log10(photplam)-2.408 
    #m = -2.5*np.log10(dn) + ABMAG_ZP
    C = photflam/Ang*(photplam*Ang)**2/c/Jy   # Jy
    if unit == "Jy": Fnu = dn*C
    elif unit == "mJy": Fnu = dn*C/mJy
    elif unit == "uJy": Fnu = dn*C/uJy
    ABmag = -2.5*np.log10(Fnu*Jy) - 48.60
    if dn_err:
        if unit == "Jy": Fnu_err = dn_err*C
        elif unit == "mJy": Fnu_err = dn_err*C/mJy
        elif unit == "uJy": Fnu_err = dn_err*C/uJy
        ABmag_err = 2.5*(Fnu_err/(Fnu*np.log(10)))
        return ABmag,ABmag_err
    else:
        return ABmag

In [4]:
def phot(image_drz,noise,photflam,photplam):
    flux = np.sum(image_drz)
    
    flux_err = noise
    #print(flux_err)
    
    if noise == 0:
        SN = 0
    else:
        SN = flux/noise
    
    if item3 == 'f225w' or item3 == 'f275w' or item3 == 'f336w' or item3 == 'f390w':
        if drop_uvis:
            mag = -99
            magerr = -99
        
        elif SN < 8:
            mag = -99
            magerr= -99
        
        else:
            mag, magerr = hst_phot(photflam, photplam, flux, dn_err=flux_err) # AB magnitude
    
    else:
        if SN < 3:
            mag = -99
            magerr = -99
    
        else:
            mag, magerr = hst_phot(photflam, photplam, flux, dn_err=flux_err) # AB magnitude
    
    return round(mag,4),round(magerr,4)

In [15]:
catalog_file = '/Users/brian.merino/Google Drive/My Drive/Clumpy_Research/galaxy_phot_v3.cat'
catalog = Table.read(catalog_file,format='ascii')

catalog.show_in_notebook()

idx,cluster,id,RA,DEC,z_clash,z_phot,z_16,z_84,f225w_mag,f225w_magerr,f275w_mag,f275w_magerr,f336w_mag,f336w_magerr,f390w_mag,f390w_magerr,f435w_mag,f435w_magerr,f475w_mag,f475w_magerr,f606w_mag,f606w_magerr,f625w_mag,f625w_magerr,f775w_mag,f775w_magerr,f814w_mag,f814w_magerr,f850lp_mag,f850lp_magerr,f105w_mag,f105w_magerr,f110w_mag,f110w_magerr,f125w_mag,f125w_magerr,f140w_mag,f140w_magerr,f160w_mag,f160w_magerr,f225w_flux,f225w_fluxerr,f275w_flux,f275w_fluxerr,f336w_flux,f336w_fluxerr,f390w_flux,f390w_fluxerr,f435w_flux,f435w_fluxerr,f475w_flux,f475w_fluxerr,f606w_flux,f606w_fluxerr,f625w_flux,f625w_fluxerr,f775w_flux,f775w_fluxerr,f814w_flux,f814w_fluxerr,f850lp_flux,f850lp_fluxerr,f105w_flux,f105w_fluxerr,f110w_flux,f110w_fluxerr,f125w_flux,f125w_fluxerr,f140w_flux,f140w_fluxerr,f160w_flux,f160w_fluxerr,z_spec
0,a209,218,22.9730375,-13.5973175,0.939,0.8484974503517151,0.8219921588897705,0.8998340964317322,21.022,0.025,-99.0,-99.0,22.711,0.068,24.107,0.13,24.274,0.114,24.094,0.068,23.743,0.033,23.941,0.068,22.948,0.034,22.67,0.02,22.769,0.06,22.679,0.015,22.58,0.008,22.393,0.012,22.3,0.009,22.298,0.014,14.164459272385136,0.3261487277091531,-99.0,-99.0,2.989509927814587,0.18723394706790525,0.826418441715795,0.09895073679244823,0.7085981269167676,0.07440130126866958,0.8363729915840239,0.05238230352683446,1.1555799554021289,0.035122839563746,0.9629417129715555,0.06030934235692439,2.4032558049332398,0.07525833347883332,3.104559588128347,0.057188101023488536,2.834001005509097,0.15661268325156918,3.078931309741611,0.042537008016982,3.37287308658869,0.024852247327167787,4.006821541822104,0.04428502825193758,4.365158322401652,0.03618413453419539,4.373206651696452,0.05639021049076047,0.939
1,a209,1020,22.978324999999998,-13.612234166666667,0.488,0.5941091179847717,0.5152083039283752,0.6043280363082886,-99.0,-99.0,-99.0,-99.0,21.628,0.032,22.892,0.055,22.664,0.034,22.573,0.022,21.845,0.007,21.672,0.011,21.32,0.01,21.218,0.007,21.196,0.018,21.105,0.005,21.089,0.003,21.036,0.004,20.947,0.003,20.869,0.005,-99.0,-99.0,-99.0,-99.0,8.105876824647362,0.23890523061861027,2.5304630494613947,0.1281853429133654,3.121763510848157,0.09775851528547144,3.3946890538007795,0.06878573161355929,6.637430704019088,0.04279309718559492,7.783949326267247,0.07886210500644963,10.764652136298333,0.09914611016122821,11.824968703831036,0.07623839065517492,12.067019095932718,0.200054435667618,13.121998990192008,0.060429038530198084,13.316803762699811,0.036795688596383526,13.982988484658337,0.051515233344450705,15.177491558612486,0.041936958974284885,16.30797361023593,0.0751009938637391,0.488
2,a209,1312,22.97045,-13.616914166666668,0.495,0.5900025367736816,0.568169355392456,0.6054204106330872,-99.0,-99.0,-99.0,-99.0,21.793,0.036,22.677,0.042,22.647,0.031,22.343,0.016,21.631,0.006,21.464,0.008,21.116,0.008,21.062,0.006,21.001,0.014,20.965,0.004,20.951,0.002,20.896,0.004,20.831,0.003,20.775,0.004,-99.0,-99.0,-99.0,-99.0,6.963057796029312,0.23087567639222017,3.0846081377571397,0.1193232216242256,3.1710274389478936,0.09053935032891668,4.195657223387249,0.061829489778459326,8.083510373900667,0.04467112916641537,9.427575085274157,0.0694649403341899,12.989726288978538,0.09571184036823951,13.652116950664434,0.07544438634818701,14.441090844161184,0.18621030682429474,14.927944095789943,0.05499657847042305,15.121678487917649,0.027855161173862338,15.907429237502162,0.05860513508820819,16.88884694790917,0.04666560866413637,17.78279410038923,0.06551423457174192,0.495
3,a209,1602,22.96975,-13.621598888888888,0.547,0.395388662815094,0.31781673431396484,0.39996740221977234,-99.0,-99.0,-99.0,-99.0,22.891,0.087,23.128,0.057,23.884,0.086,22.976,0.026,22.402,0.01,22.288,0.016,22.081,0.016,22.072,0.013,22.152,0.037,22.227,0.011,22.268,0.007,22.304,0.012,22.277,0.01,22.276,0.015,-99.0,-99.0,-99.0,-99.0,2.5327947656899945,0.20295274639397126,2.036104201131967,0.10689331253384562,1.0148456628180937,0.0803848362243744,2.3420706947938243,0.05608529751313286,3.973745050614623,0.036599544467

In [6]:
'''
filter_list = ['f225w','f275w','f336w','f390w','f435w','f475w',\
               'f606w','f625w','f775w','f814w','f850lp','f105w',\
               'f110w','f125w','f140w','f160w']
'''
filter_list = ['f435w','f475w','f606w','f625w','f775w','f814w','f850lp','f105w',\
               'f110w','f125w','f140w','f160w']

root  = '/Users/brian.merino/Google Drive/My Drive/Clumpy_Research/galaxy_masks/'
root2 = '/Users/brian.merino/Google Drive/My Drive/Clumpy_Research/clump_masks/'
ext = 0
drop_uvis = 1

In [19]:
new_galaxy_cat_path = '/Users/brian.merino/Google Drive/My Drive/Clumpy_Research/dendro_galaxy_phot_v2.cat'
new_galaxy_catalog = Table.read(new_galaxy_cat_path,format='ascii')

new_galaxy_catalog.show_in_notebook()

idx,cluster,id,ra,dec,z_clash,z_phot,z_16,z_84,z_spec,z_note,f225w_mag,f225w_magerr,f275w_mag,f275w_magerr,f336w_mag,f336w_magerr,f390w_mag,f390w_magerr,f435w_mag,f435w_magerr,f475w_mag,f475w_magerr,f606w_mag,f606w_magerr,f625w_mag,f625w_magerr,f775w_mag,f775w_magerr,f814w_mag,f814w_magerr,f850lp_mag,f850lp_magerr,f105w_mag,f105w_magerr,f110w_mag,f110w_magerr,f125w_mag,f125w_magerr,f140w_mag,f140w_magerr,f160w_mag,f160w_magerr
0,a209,218,22.97304,-13.59732,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,24.2165,0.0899,23.6698,0.0382,23.6165,0.0303,23.4329,0.0421,22.6195,0.0375,22.5406,0.0124,22.4963,0.0262,22.2952,0.011,22.2129,0.0088,22.0618,0.0095,21.9148,0.0072,21.9113,0.0077
1,a209,1020,22.97832,-13.61223,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,22.7398,0.0225,22.5927,0.0138,21.9523,0.0068,21.7782,0.009,21.4682,0.008,21.4329,0.0046,21.3528,0.0091,21.2266,0.0043,21.1909,0.0035,21.1454,0.0044,21.0741,0.0036,20.9915,0.0034
2,a209,1312,22.97045,-13.61691,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,22.3346,0.0348,22.4681,0.0161,21.8448,0.0082,21.747,0.0118,21.445,0.0102,21.4316,0.0058,21.3961,0.0118,21.3433,0.006,21.3139,0.0049,21.2987,0.0063,21.2861,0.0054,21.2529,0.0055
3,a209,1602,22.96975,-13.6216,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,23.075,0.0211,22.4782,0.013,22.4081,0.0183,22.228,0.0184,22.264,0.0097,22.269,0.0196,22.1545,0.0092,22.1568,0.0078,22.1456,0.0101,22.1118,0.0085,22.0692,0.0085
4,a209,1669,22.98161,-13.62112,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,21.7534,0.0079,21.7019,0.0058,21.5118,0.0042,21.414,0.0059,21.0817,0.0053,21.0241,0.0029,20.887,0.0055,20.9516,0.0031,20.8288,0.0024,20.7407,0.0028,20.7435,0.0024,20.8488,0.0028
5,a209,1671,22.97968,-13.62333,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,22.1082,0.0106,21.9936,0.0072,21.3991,0.0072,21.2294,0.0047,21.0548,0.005,20.9961,0.0027,20.9132,0.0052,20.8932,0.0028,20.8714,0.0023,20.8631,0.003,20.7974,0.0025,20.7684,0.0025
6,a209,1708,22.96176,-13.62408,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,24.4032,0.0622,24.5193,0.054,24.1815,0.0331,24.0631,0.0457,23.5307,0.0345,23.3792,0.018,23.3104,0.0343,23.2072,0.0173,23.1391,0.0137,23.0735,0.0169,23.0872,0.0149,23.0795,0.0155
7,a209,1772,22.98063,-13.62598,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,25.0993,0.0689,25.1576,0.0548,24.9389,0.0377,25.0021,0.0625,24.5357,0.0483,24.4484,0.0264,24.3373,0.0513,24.3229,0.0385,24.4748,0.0264,-99.0,-99.0,-99.0,-99.0,24.5692,0.0408
8,a383,58,42.01558,-3.50832,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,22.9199,0.0163,22.487,0.0095,21.8502,0.0044,21.7344,0.0057,21.62,0.0062,21.53,0.0034,21.3995,0.0063,21.4103,0.0062,-99.0,-99.0,21.2943,0.0042,21.2374,0.0046,21.198,0.005
9,a383,128,42.0005,-3.51168,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,24.2129,0.0483,24.4188,0.0493,24.054,0.0291,24.293,0.0519,23.5967,0.0334,23.3399,0.0156,23.3024,0.0316,23.3171,0.0303,23.2252,0.0138,-99.0,-99.0,23.242,0.029,23.2307,0.0257


In [21]:
#for i in range(0,len(catalog[:8])):
#for q in range(0,len(catalog)):
for q in range(140,141):
    filter_dict = {}
    cluster = catalog['cluster'][q]
    ID = catalog['id'][q]
    ra  = catalog['RA'][q]
    dec = catalog['DEC'][q]
    
    if cluster == 'a611' or cluster =='macs0744' or cluster =='macs0329':
        filter_list = ['f105w','f110w','f125w','f140w','f160w',\
                       'f225w','f275w','f336w','f390w','f435w','f475w',\
                       'f606w','f775w','f814w','f850lp']

    elif cluster == 'macs1423':
        filter_list = ['f105w','f110w','f125w','f140w','f160w',\
                       'f225w','f275w','f336w','f390w','f435w','f475w',\
                       'f606w','f775w','f850lp']
        
    else:
        filter_list = ['f225w','f275w','f336w','f390w','f435w','f475w',\
               'f606w','f625w','f775w','f814w','f850lp','f105w',\
               'f110w','f125w','f140w','f160w']
    
    new_galaxy_catalog['cluster'][q]      = cluster
    new_galaxy_catalog['id'][q]           = ID  
    new_galaxy_catalog['ra'][q]           = round(ra,5)
    new_galaxy_catalog['dec'][q]          = round(dec,5)
    new_galaxy_catalog['z_phot'][q]       = -99.99
    new_galaxy_catalog['z_16'][q]         = -99.99
    new_galaxy_catalog['z_84'][q]         = -99.99
    new_galaxy_catalog['z_spec'][q]       = -99.99
    new_galaxy_catalog['z_note'][q]       = -99
        
    for item3 in filter_list:
        image_path = root + '%s-%s-%s_galaxy.fits'%(item3,cluster,ID)
        wht_path   = root + '%s-%s-%s_galaxy_wht.fits'%(item3,cluster,ID)
        
        print('\n Opening image: %s \n'%(image_path))

        pf = fits.open(image_path)
        image    = pf[ext].data
        head     = pf[ext].header
        photflam = head['photflam']
        photplam = head['photplam']
        photbw   = head['photbw']

        pf_wht = fits.open(wht_path)
        wht = pf_wht[ext].data #inverse variance weight images

        quotient = 1/wht
        
        for i in range(0,quotient.shape[0]):
            for j in range(0,quotient.shape[1]):
                if np.isinf(quotient[i][j]):
                    quotient[i][j]=0

        SUM = np.sum(quotient)

        noise = np.sqrt(SUM)

        wcs = WCS(head)
        wcs.sip = None
        
        print(item3)
        new_galaxy_catalog['%s_mag'%(item3)][q],new_galaxy_catalog['%s_magerr'%(item3)][q] = phot(image,noise,photflam,photplam)
    


 Opening image: /Users/brian.merino/Google Drive/My Drive/Clumpy_Research/galaxy_masks/f225w-rxj2129-1546_galaxy.fits 

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please 

<ipython-input-21-a41bbab56ed3>:52: RuntimeWarning: divide by zero encountered in true_divide
  quotient = 1/wht


f336w

 Opening image: /Users/brian.merino/Google Drive/My Drive/Clumpy_Research/galaxy_masks/f390w-rxj2129-1546_galaxy.fits 

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency p

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
f125w

 Opening imag

In [23]:
new_galaxy_catalog.write(new_galaxy_cat_path,format='ascii',overwrite=True)

In [22]:
new_galaxy_catalog.show_in_notebook()

idx,cluster,id,ra,dec,z_clash,z_phot,z_16,z_84,z_spec,z_note,f225w_mag,f225w_magerr,f275w_mag,f275w_magerr,f336w_mag,f336w_magerr,f390w_mag,f390w_magerr,f435w_mag,f435w_magerr,f475w_mag,f475w_magerr,f606w_mag,f606w_magerr,f625w_mag,f625w_magerr,f775w_mag,f775w_magerr,f814w_mag,f814w_magerr,f850lp_mag,f850lp_magerr,f105w_mag,f105w_magerr,f110w_mag,f110w_magerr,f125w_mag,f125w_magerr,f140w_mag,f140w_magerr,f160w_mag,f160w_magerr
0,a209,218,22.97304,-13.59732,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,24.2165,0.0899,23.6698,0.0382,23.6165,0.0303,23.4329,0.0421,22.6195,0.0375,22.5406,0.0124,22.4963,0.0262,22.2952,0.011,22.2129,0.0088,22.0618,0.0095,21.9148,0.0072,21.9113,0.0077
1,a209,1020,22.97832,-13.61223,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,22.7398,0.0225,22.5927,0.0138,21.9523,0.0068,21.7782,0.009,21.4682,0.008,21.4329,0.0046,21.3528,0.0091,21.2266,0.0043,21.1909,0.0035,21.1454,0.0044,21.0741,0.0036,20.9915,0.0034
2,a209,1312,22.97045,-13.61691,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,22.3346,0.0348,22.4681,0.0161,21.8448,0.0082,21.747,0.0118,21.445,0.0102,21.4316,0.0058,21.3961,0.0118,21.3433,0.006,21.3139,0.0049,21.2987,0.0063,21.2861,0.0054,21.2529,0.0055
3,a209,1602,22.96975,-13.6216,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,23.075,0.0211,22.4782,0.013,22.4081,0.0183,22.228,0.0184,22.264,0.0097,22.269,0.0196,22.1545,0.0092,22.1568,0.0078,22.1456,0.0101,22.1118,0.0085,22.0692,0.0085
4,a209,1669,22.98161,-13.62112,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,21.7534,0.0079,21.7019,0.0058,21.5118,0.0042,21.414,0.0059,21.0817,0.0053,21.0241,0.0029,20.887,0.0055,20.9516,0.0031,20.8288,0.0024,20.7407,0.0028,20.7435,0.0024,20.8488,0.0028
5,a209,1671,22.97968,-13.62333,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,22.1082,0.0106,21.9936,0.0072,21.3991,0.0072,21.2294,0.0047,21.0548,0.005,20.9961,0.0027,20.9132,0.0052,20.8932,0.0028,20.8714,0.0023,20.8631,0.003,20.7974,0.0025,20.7684,0.0025
6,a209,1708,22.96176,-13.62408,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,24.4032,0.0622,24.5193,0.054,24.1815,0.0331,24.0631,0.0457,23.5307,0.0345,23.3792,0.018,23.3104,0.0343,23.2072,0.0173,23.1391,0.0137,23.0735,0.0169,23.0872,0.0149,23.0795,0.0155
7,a209,1772,22.98063,-13.62598,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,25.0993,0.0689,25.1576,0.0548,24.9389,0.0377,25.0021,0.0625,24.5357,0.0483,24.4484,0.0264,24.3373,0.0513,24.3229,0.0385,24.4748,0.0264,-99.0,-99.0,-99.0,-99.0,24.5692,0.0408
8,a383,58,42.01558,-3.50832,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,22.9199,0.0163,22.487,0.0095,21.8502,0.0044,21.7344,0.0057,21.62,0.0062,21.53,0.0034,21.3995,0.0063,21.4103,0.0062,-99.0,-99.0,21.2943,0.0042,21.2374,0.0046,21.198,0.005
9,a383,128,42.0005,-3.51168,-99.99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,24.2129,0.0483,24.4188,0.0493,24.054,0.0291,24.293,0.0519,23.5967,0.0334,23.3399,0.0156,23.3024,0.0316,23.3171,0.0303,23.2252,0.0138,-99.0,-99.0,23.242,0.029,23.2307,0.0257


### Clump photometry code is below this cell

In [16]:
new_clump_cat_path = '/Users/brian.merino/Google Drive/My Drive/Clumpy_Research/dendro_clump_phot_v2.cat'
new_clump_catalog = Table.read(new_clump_cat_path,format='ascii')

new_clump_catalog.show_in_notebook()

idx,cluster,id,clump_num,ra,dec,z_clash,z_phot,z_16,z_84,z_spec,z_note,f225w_mag,f225w_magerr,f275w_mag,f275w_magerr,f336w_mag,f336w_magerr,f390w_mag,f390w_magerr,f435w_mag,f435w_magerr,f475w_mag,f475w_magerr,f606w_mag,f606w_magerr,f625w_mag,f625w_magerr,f775w_mag,f775w_magerr,f814w_mag,f814w_magerr,f850lp_mag,f850lp_magerr,f105w_mag,f105w_magerr,f110w_mag,f110w_magerr,f125w_mag,f125w_magerr,f140w_mag,f140w_magerr,f160w_mag,f160w_magerr
0,a209,218,1,22.97304,-13.59732,0.939,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,24.7673,0.0416,24.7886,0.0323,24.6922,0.0242,24.5853,0.0369,23.9363,0.0225,23.9133,0.0135,23.9581,0.0304,23.9014,0.014,23.9286,0.0125,23.8126,0.0138,23.7512,0.0114,23.7774,0.0126
1,a209,1020,1,22.97832,-13.61223,0.488,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,25.1285,0.0549,25.0671,0.036,24.4828,0.0192,24.3454,0.0276,24.1402,0.0262,24.1592,0.0154,24.1679,0.0323,24.0704,0.0163,24.0643,0.0139,24.0273,0.0171,24.0262,0.0149,23.9451,0.0145
2,a209,1312,1,22.97045,-13.61691,0.495,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,27.2648,0.1193,26.7963,0.0883,26.8144,0.1236,26.6811,0.1267,26.7874,0.0668,27.57,0.293,27.2173,0.1116,27.4309,0.1105,27.7434,0.1994,27.8398,0.1921,27.28,0.1179
3,a209,1312,2,22.97045,-13.61691,0.495,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,24.9012,0.1049,24.8325,0.0213,24.6602,0.0188,24.6018,0.0333,24.4451,0.0264,24.4903,0.0155,24.5561,0.0331,24.5932,0.0187,24.6354,0.0166,24.7176,0.0231,24.6662,0.0195,24.6114,0.0192
4,a209,1602,1,22.96975,-13.6216,0.547,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,25.9175,0.0595,25.8538,0.0555,25.3782,0.0231,25.2237,0.0302,25.0365,0.0321,25.1021,0.0192,25.1549,0.0447,25.1331,0.0228,25.1264,0.0191,25.2301,0.0272,25.1596,0.0222,25.1029,0.0222
5,a209,1602,2,22.96975,-13.6216,0.547,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,26.9872,0.0767,26.5215,0.0535,26.484,0.083,26.1812,0.0782,26.2375,0.0343,26.7338,0.1274,26.4813,0.0472,26.597,0.0451,26.5805,0.0567,26.6051,0.0501,26.5429,0.0498
6,a209,1669,1,22.98161,-13.62112,0.796,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,24.9957,0.0122,25.0553,0.0101,24.7616,0.0072,24.6768,0.0096,24.3934,0.0105,24.2567,0.0048,24.1638,0.0103,24.4466,0.0071,24.3663,0.0055,24.3105,0.0067,24.3401,0.0059,24.4279,0.0069
7,a209,1669,2,22.98161,-13.62112,0.796,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,24.1518,0.0087,24.2056,0.0072,24.0185,0.0059,23.9544,0.0081,23.6253,0.0064,23.585,0.0039,23.7063,0.0098,23.7623,0.0055,23.6978,0.0047,23.6269,0.0052,23.6589,0.0046,23.7193,0.0053
8,a209,1671,1,22.97968,-13.62333,0.443,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,25.6676,0.0301,25.7107,0.0261,25.403,0.0187,25.3176,0.022,25.196,0.0263,25.1923,0.0146,25.2972,0.0381,25.3206,0.0187,25.3559,0.0163,25.3604,0.0215,25.3141,0.0186,25.3115,0.0185
9,a209,1708,1,22.96176,-13.62408,0.817,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,26.247,0.0756,26.4445,0.0784,26.1772,0.0487,26.1722,0.0697,25.7213,0.0627,25.7315,0.0354,25.959,0.0887,25.7742,0.042,25.7832,0.0352,25.7144,0.0431,25.7424,0.0386,25.8657,0.0459


In [23]:
#for q in range(len(new_galaxy_catalog)):
clump_num_dict = {}

#There are only 17 sources in this catalog so far
#sources = 17
#sources = len(catalog)
for q in range(0,len(catalog)):
    cluster = catalog['cluster'][q]
    ID = catalog['id'][q]
    
    #Use glob to find out how many clumps are in each galaxy
    clump_string = glob.glob(root[:-13]+'clump_masks/*-%s-%s_clump_*.fits'%(cluster,ID))
    sci = []
    wht_list = []

    for t2 in range(len(clump_string)):
        if clump_string[t2][-8:] == 'wht.fits':
            wht_list.append(clump_string[t2])
        else:
            sci.append(clump_string[t2])
    
    number_of_clumps = int(sci[-1][-6])
    if number_of_clumps == 0:
        number_of_clumps = int(sci[-1][-7:-5])
    
    print("Check this value")
    print("%s %s"%(cluster,ID))
    print("There are %i clumps\n"%(number_of_clumps))
    
    clump_num_dict['%s-%s'%(cluster,ID)] = number_of_clumps
#print(clump_num_dict)

Check this value
macs1206 253
There are 3 clumps

Check this value
macs1720 297
There are 1 clumps

Check this value
macs1720 344
There are 1 clumps

Check this value
rxj1347 1507
There are 1 clumps

Check this value
rxj2129 597
There are 4 clumps

Check this value
rxj2129 1359
There are 6 clumps



In [28]:
base = '/Users/brian.merino/Google Drive/My Drive/Clumpy_Research/clump_masks/'
counter = 0
for q in range(0,len(catalog)):
    cluster = catalog['cluster'][q]
    ID = catalog['id'][q]
    ra  = catalog['RA'][q]
    dec = catalog['DEC'][q]
    z_clash = catalog['z_clash'][q]
    
    if cluster == 'a611' or cluster =='macs0744' or cluster =='macs0329':
        filter_list = ['f105w','f110w','f125w','f140w','f160w',\
                       'f225w','f275w','f336w','f390w','f435w','f475w',\
                       'f606w','f775w','f814w','f850lp']

    elif cluster == 'macs1423':
        filter_list = ['f105w','f110w','f125w','f140w','f160w',\
                       'f225w','f275w','f336w','f390w','f435w','f475w',\
                       'f606w','f775w','f850lp']
        
    else:
        filter_list = ['f225w','f275w','f336w','f390w','f435w','f475w',\
               'f606w','f625w','f775w','f814w','f850lp','f105w',\
               'f110w','f125w','f140w','f160w']
    
    number = clump_num_dict['%s-%s'%(cluster,ID)]
    
        
    print('#################')
    print(cluster,ID,number)
    print('#################')    
    
    filter_dict = {}
    
    for n in range(1,number+1):
        for item3 in filter_list:
            image_path = base + '%s-%s-%s_clump_%i.fits'%(item3,cluster,ID,n)
            wht_path   = base + '%s-%s-%s_clump_%i_wht.fits'%(item3,cluster,ID,n)

            clump_num = int(sci[-1][-6])
            if clump_num == 0:
                clump_num = int(sci[-1][-7:-5])

            pf = fits.open(image_path)
            image    = pf[ext].data
            head     = pf[ext].header
            photflam = head['photflam']
            photplam = head['photplam']
            photbw   = head['photbw']

            pf_wht = fits.open(wht_path)
            wht = pf_wht[ext].data #inverse variance weight images
            quotient = 1/wht

            for i in range(0,quotient.shape[0]):
                for j in range(0,quotient.shape[1]):
                    if np.isinf(quotient[i][j]):
                        quotient[i][j]=0

            SUM = np.sum(quotient)
            noise = np.sqrt(SUM)

            wcs = WCS(head)
            wcs.sip = None
        
            filter_dict['%s_clump_%s_mag'%(item3,n)],filter_dict['%s_clump_%s_magerr'%(item3,n)] = phot(image,noise,photflam,photplam)
            
    for value in range(1,number+1):
        new_clump_catalog['clump_num'][counter]    = value
        new_clump_catalog['cluster'][counter]      = cluster
        new_clump_catalog['id'][counter]           = ID  
        new_clump_catalog['ra'][counter]           = round(ra,5)
        new_clump_catalog['dec'][counter]          = round(dec,5)
        new_clump_catalog['z_clash'][counter]      = z_clash
        new_clump_catalog['z_phot'][counter]       = -99.99
        new_clump_catalog['z_16'][counter]         = -99.99
        new_clump_catalog['z_84'][counter]         = -99.99
        new_clump_catalog['z_spec'][counter]       = -99.99
        new_clump_catalog['z_note'][counter]       = -99
        
        for item2 in filter_list:
            new_clump_catalog['%s_mag'%(item2)][counter]    = filter_dict['%s_clump_%s_mag'%(item2,value)]
            new_clump_catalog['%s_magerr'%(item2)][counter] = filter_dict['%s_clump_%s_magerr'%(item2,value)]
        counter+=1

#################
macs1206 253 3
#################
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

    

<ipython-input-28-57a9560d8205>:53: RuntimeWarning: divide by zero encountered in true_divide
  quotient = 1/wht


INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
INFO: 
             

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
INFO: 
             

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
INFO: 
             

...

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
INFO: 
             

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
INFO: 
             

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
INFO: 
             

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
INFO: 
             

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
INFO: 
             

In [29]:
new_clump_catalog.write(new_clump_cat_path,format='ascii',overwrite=True)

In [30]:
new_clump_catalog.show_in_notebook()

idx,cluster,id,clump_num,ra,dec,z_clash,z_phot,z_16,z_84,z_spec,z_note,f225w_mag,f225w_magerr,f275w_mag,f275w_magerr,f336w_mag,f336w_magerr,f390w_mag,f390w_magerr,f435w_mag,f435w_magerr,f475w_mag,f475w_magerr,f606w_mag,f606w_magerr,f625w_mag,f625w_magerr,f775w_mag,f775w_magerr,f814w_mag,f814w_magerr,f850lp_mag,f850lp_magerr,f105w_mag,f105w_magerr,f110w_mag,f110w_magerr,f125w_mag,f125w_magerr,f140w_mag,f140w_magerr,f160w_mag,f160w_magerr
0,macs1206,253,1,181.54457,-8.78759,-99.99,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,26.5053,0.0705,26.5642,0.0745,25.6314,0.0249,25.4233,0.0249,24.4742,0.0131,24.2769,0.0079,23.9068,0.0108,23.7215,0.0053,23.6013,0.0036,23.4768,0.0046,23.3711,0.0037,23.2825,0.0035
1,macs1206,253,2,181.54457,-8.78759,-99.99,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,27.2569,0.0949,26.9524,0.0715,26.2313,0.0275,25.938,0.0299,25.1379,0.0156,24.9282,0.009,24.591,0.0129,24.4087,0.0067,24.1874,0.0041,24.0256,0.0053,23.8846,0.0041,23.7651,0.0038
2,macs1206,253,3,181.54457,-8.78759,-99.99,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,25.3357,0.0243,25.3444,0.0222,25.1789,0.0425,24.971,0.0178,24.396,0.0135,24.2771,0.007,24.0069,0.0111,24.0014,0.0068,23.8593,0.0046,23.7904,0.0061,23.6808,0.0049,23.5432,0.0045
3,macs1720,297,1,260.05667,35.62135,-99.99,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,25.5163,0.0296,25.4677,0.0201,25.0999,0.0119,25.0384,0.0184,25.0771,0.0213,25.0129,0.0118,24.9508,0.0288,24.9354,0.0148,24.8781,0.0093,24.8043,0.0136,24.7444,0.0106,24.7249,0.0116
4,macs1720,297,2,260.05667,35.62135,-99.99,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,22.8155,0.0042,22.6528,0.0028,22.1199,0.0015,21.9683,0.0018,21.8032,0.002,21.7422,0.0011,21.6662,0.0024,21.5937,0.0012,21.5259,0.0008,21.4307,0.0011,21.2691,0.0008,21.1131,0.0008
5,macs1720,297,3,260.05667,35.62135,-99.99,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,25.8351,0.0205,25.6285,0.0128,25.0203,0.0061,24.855,0.0077,24.5724,0.0077,24.5299,0.0044,24.4339,0.009,24.3715,0.005,24.2447,0.0029,24.1452,0.0041,24.0122,0.003,23.8517,0.003
6,macs1720,297,4,260.05667,35.62135,-99.99,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,24.2819,0.0157,24.0759,0.009,23.418,0.0042,23.2595,0.0058,23.0441,0.0053,23.0123,0.0031,22.927,0.0064,22.8178,0.0033,22.7676,0.0021,22.6826,0.0031,22.5843,0.0023,22.4576,0.0023
7,macs1720,297,5,260.05667,35.62135,-99.99,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,26.2501,0.0258,26.1338,0.021,25.9295,0.0151,25.8126,0.0185,25.8815,0.0231,25.8365,0.0143,25.7295,0.028,25.8409,0.017,25.7984,0.0115,25.7717,0.0167,25.6704,0.0124,25.5927,0.0134
8,macs1720,297,6,260.05667,35.62135,-99.99,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,26.226,0.0516,25.9168,0.0283,24.9182,0.0092,24.7027,0.0116,24.1998,0.0095,24.0407,0.0044,23.7038,0.0078,23.5501,0.004,23.2875,0.002,23.1503,0.0028,22.9491,0.0019,22.8488,0.002
9,macs1720,297,7,260.05667,35.62135,-99.99,-99,-99,-99,-99,-99,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,24.6798,0.0175,24.6447,0.0125,24.3556,0.0088,24.2447,0.011,24.2439,0.0131,24.1615,0.0076,24.0698,0.0156,24.138,0.0093,24.0275,0.0057,24.0204,0.0087,23.9224,0.0066,23.8999,0.0072
